## Import Model

In [1]:
# CELL 1 — Setup: imports, environment, OpenAI client, model name

import os
import json
import time
import math
import re

import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# Load .env so OPENAI_API_KEY is available as an environment variable
load_dotenv()

# Create OpenAI client (picks up OPENAI_API_KEY automatically)
client = OpenAI()

# Reasoning model to use
MODEL_NAME = "o3-mini"


## Definitions and Prompt

In [ ]:
# CELL 2 — Label definitions and helper to build text for the prompt

LABEL_DEFINITIONS = {
    "SUPPORT_VALIDATION": (
        "The commenter clearly validates OP (says it was assault, "
        "not their fault, they deserve care, or that what happened was wrong.)"
    ),
    "SUPPORT_SHARED_STORY": (
        "The commenter shares their OWN similar experience in a supportive way, "
        "to show OP they are not alone and to normalize OP's feelings."
    ),
    "GENTLE_ADVICE": (
        "The commenter gives gentle, non-judgmental suggestions (e.g., talk to a friend, "
        "therapist, doctor, or set boundaries) without blaming OP."  
    ),
    "BLAMING_OP": (
        "The commenter implies OP is at fault or did something wrong, "
        "e.g., 'you should have...', 'why did you...', or suggests OP caused the assault."
    ),
    "MINIMIZING_OP_EXPERIENCE": (
        "The commenter downplays or dismisses what happened to OP, e.g., "
        "'not that bad', 'doesn't really count', or suggests OP is overreacting."
    ),
    "NEUTRAL_OR_INFO": (
        "Mostly neutral or informational: explaining terms, giving general info, "
        "or logistics/resources without clear emotional stance."
    ),
}

STANCE_DEFINITIONS = """
stance_towards_op should be exactly ONE of:
- SUPPORTIVE: at least one supportive label (SUPPORT_VALIDATION, SUPPORT_SHARED_STORY, GENTLE_ADVICE) and NO BLAMING_OP or MINIMIZING_OP_EXPERIENCE.
- NEGATIVE: at least one of BLAMING_OP or MINIMIZING_OP_EXPERIENCE and NO supportive labels.
- MIXED: at least one supportive label AND at least one of BLAMING_OP or MINIMIZING_OP_EXPERIENCE.
- NEUTRAL: only NEUTRAL_OR_INFO is present.
- UNCLEAR: none of the labels apply or the stance is hard to determine.
""".strip()


def build_label_definitions_text():
    lines = []
    for label, desc in LABEL_DEFINITIONS.items():
        lines.append(f"- {label}: {desc}")
    return "\n".join(lines)


LABEL_DEFINITIONS_TEXT = build_label_definitions_text()
